In [1]:
import pandas as pd
import numpy as np

## Configuration

In [2]:
TARGET = "pollutant"
DATA_PATH = "datasets"

API_URLS = [
    "http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first",
    "http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second",
    "http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third"
]

DATASET_DTYPES = {
    "CITY ID": "object",
    "CONTINENT": "object",
    "City": "object",
    "DAY": "int8",
    "DAY WITH FOGS": "int8",
    "EPRTRAnnexIMainActivityCode": "object",
    "EPRTRAnnexIMainActivityLabel": "object",
    "EPRTRSectorCode": "int8",
    "FacilityInspireID": "object",
    "MONTH": "int8",
    "REPORTER NAME": "object",
    "avg_temp": "float64",
    "avg_wind_speed": "float64",
    "countryName": "object",
    "eprtrSectorName": "object",
    "facilityName": "object",
    "max_temp": "float64",
    "max_wind_speed": "float64",
    "min_temp": "float64",
    "min_wind_speed": "float64",
    "pollutant": "object",
    "reportingYear": "int8",
    "targetRelease": "object"
}

## Classes

In [3]:
class ApiDataConsumer:

    def __init__(self, urls, dtypes = None):
        self.urls = urls
        self.dtypes = dtypes

    def get_api_df(self, verbose: bool = True):
        if verbose:
            print("Getting data from API")
        api_df = None
        for url in self.urls:
            json_df = pd.read_json(url, dtype=self.dtypes)
            json_df = json_df.drop("", axis=1)

            if verbose:
                print(f"url: {url} json_df.shape: {json_df.shape}")

            if api_df is not None:
                api_df = pd.concat([api_df, json_df])
            else:
                api_df = json_df

        return api_df

## Load data

In [4]:
apiDataConsumer = ApiDataConsumer(API_URLS, DATASET_DTYPES)

In [5]:
%%time
train1_df = pd.read_csv(f"{DATA_PATH}/train1.csv", dtype=DATASET_DTYPES)
print('* train1 loaded... shape:', train1_df.shape)

train2_df = pd.read_csv(f"{DATA_PATH}/train2.csv", dtype=DATASET_DTYPES, sep=";")
print('* train2 loaded... shape:', train2_df.shape)

api_df = apiDataConsumer.get_api_df(verbose=False)
print('* api_df loaded... shape:', api_df.shape)

* train1 loaded... shape: (18563, 21)
* train2 loaded... shape: (18564, 21)
* api_df loaded... shape: (28501, 23)
Wall time: 6.51 s


In [6]:
raw_df = pd.concat([train1_df, train2_df, api_df], axis=0)
print('* raw_df shape:', raw_df.shape)
raw_df.head()

* raw_df shape: (65628, 23)


,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,pollutant,reportingYear,MONTH,...,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID,EPRTRAnnexIMainActivityCode,EPRTRSectorCode
0,Germany,Mineral industry,Installations for the production of cement cli...,https://registry.gdi-de.org/id/de.ni.mu/062217...,Holcim (Deutschland) GmbH Werk Höver,Sehnde,AIR,Carbon dioxide (CO2),-33,10,...,14.312541,21.419106,2.864895,4.924169,9.688206,2,Mr. Jacob Ortega,7cdb5e74adcb2ffaa21c1b61395a984f,NaN,NaN
1,Italy,Mineral industry,Installations for the production of cement cli...,IT.CAED/240602021.FACILITY,Stabilimento di Tavernola Bergamasca,TAVERNOLA BERGAMASCA,AIR,Nitrogen oxides (NOX),-30,9,...,19.368166,21.756389,5.462839,7.864403,12.023521,1,Ashlee Serrano,cd1dbabbdba230b828c657a9b19a8963,NaN,NaN
2,Spain,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,ES.CAED/001966000.FACILITY,COMPLEJO MEDIOAMBIENTAL DE ZURITA,PUERTO DEL ROSARIO,AIR,Methane (CH4),-29,2,...,14.701985,17.103930,1.511201,4.233438,8.632193,2,Vincent Kemp,5011e3fa1436d15b34f1287f312fbada,NaN,NaN
3,Czechia,Energy sector,Thermal power stations and other combustion in...,CZ.MZP.U422/CZ34736841.FACILITY,Elektrárny Prunéřov,Kadaň,AIR,Nitrogen oxides (NOX),-36,8,...,16.122584,17.537184,10.970301,10.298348,15.179215,0,Carol Gray,37a6d7a71c4f7c2469e4f01b70dd90c2,NaN,NaN
4,Finland,Waste and wastewater management,Urban waste-water treatment plants,http://paikkatiedot.fi/so/1002031/pf/Productio...,"TAMPEREEN VESI LIIKELAITOS, VIINIKANLAHDEN JÄT...",Tampere,AIR,Methane (CH4),-30,12,...,20.201604,21.536012,11.772039,11.344078,16.039004,2,Blake Ford,471fe554e1c62d1b01cc8e4e5076c61a,NaN,NaN


## Preprocessing

In [7]:
prep_df = raw_df.copy(deep=True)

In [8]:
prep_df

,countryName,eprtrSectorName,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,pollutant,reportingYear,MONTH,...,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID,EPRTRAnnexIMainActivityCode,EPRTRSectorCode
0,Germany,Mineral industry,Installations for the production of cement cli...,https://registry.gdi-de.org/id/de.ni.mu/062217...,Holcim (Deutschland) GmbH Werk Höver,Sehnde,AIR,Carbon dioxide (CO2),-33,10,...,14.312541,21.419106,2.864895,4.924169,9.688206,2,Mr. Jacob Ortega,7cdb5e74adcb2ffaa21c1b61395a984f,NaN,NaN
1,Italy,Mineral industry,Installations for the production of cement cli...,IT.CAED/240602021.FACILITY,Stabilimento di Tavernola Bergamasca,TAVERNOLA BERGAMASCA,AIR,Nitrogen oxides (NOX),-30,9,...,19.368166,21.756389,5.462839,7.864403,12.023521,1,Ashlee Serrano,cd1dbabbdba230b828c657a9b19a8963,NaN,NaN
2,Spain,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,ES.CAED/001966000.FACILITY,COMPLEJO MEDIOAMBIENTAL DE ZURITA,PUERTO DEL ROSARIO,AIR,Methane (CH4),-29,2,...,14.701985,17.103930,1.511201,4.233438,8.632193,2,Vincent Kemp,5011e3fa1436d15b34f1287f312fbada,NaN,NaN
3,Czechia,Energy sector,Thermal power stations and other combustion in...,CZ.MZP.U422/CZ34736841.FACILITY,Elektrárny Prunéřov,Kadaň,AIR,Nitrogen oxides (NOX),-36,8,...,16.122584,17.537184,10.970301,10.298348,15.179215,0,Carol Gray,37a6d7a71c4f7c2469e4f01b70dd90c2,NaN,NaN
4,Finland,Waste and wastewater management,Urban waste-water treatment plants,http://paikkatiedot.fi/so/1002031/pf/Productio...,"TAMPEREEN VESI LIIKELAITOS, VIINIKANLAHDEN JÄT...",Tampere,AIR,Methane (CH4),-30,12,...,20.201604,21.536012,11.772039,11.344078,16.039004,2,Blake Ford,471fe554e1c62d1b01cc8e4e5076c61a,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9496,Cyprus,Energy sector,Thermal power stations and other combustion in...,CY.CAED/0030030000.FACILITY,"Electricity Authority of Cyprus, Vassilikos Po...",LARNAKA,AIR,Carbon dioxide (CO2),-40,1,...,18.556476,22.852530,13.345801,12.410783,17.148327,0,Tammy Faulkner,2d4776365b33d5f1be53ea4606e2c79c,1(c),1.0
9497,Finland,Energy sector,Thermal power stations and other combustion in...,http://paikkatiedot.fi/so/1002031/pf/Productio...,"Turun Seudun Energiantuotanto Oy, Naantalin vo...",Naantali,AIR,Nitrogen oxides (NOX),-40,12,...,14.461703,20.553781,3.820281,3.763833,5.657107,0,Dr. Courtney Bryant,020b11bf06b96aae1dd910a56674a8aa,1(c),1.0
9498,Slovenia,Waste and wastewater management,Landfills (excluding landfills of inert waste ...,SI.ARSO/000000037.FACILITY,"Javne službe Ptuj, Odlagališče nenevarnih odpa...",Ptuj,AIR,Methane (CH4),-38,8,...,16.688049,20.411498,17.285365,18.349798,21.538441,2,William Greer,84afdc8367dfd9124e8b8f994e986fe9,5(d),5.0
9499,Italy,Mineral industry,Underground mining and related operations,IT.CAED/850592002.FACILITY,Centro Olio Val d'Agri,VIGGIANO,AIR,Nitrogen oxides (NOX),-34,1,...,16.144091,22.647192,6.387199,6.176238,9.269076,0,Leonard Roberts,09ad69bcf41256f40be3314a33e0438c,3(a),3.0


In [9]:
prep_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65628 entries, 0 to 9500
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   countryName                   65628 non-null  object 
 1   eprtrSectorName               65628 non-null  object 
 2   EPRTRAnnexIMainActivityLabel  65628 non-null  object 
 3   FacilityInspireID             65628 non-null  object 
 4   facilityName                  65628 non-null  object 
 5   City                          65628 non-null  object 
 6   targetRelease                 65628 non-null  object 
 7   pollutant                     65628 non-null  object 
 8   reportingYear                 65628 non-null  int8   
 9   MONTH                         65628 non-null  int8   
 10  DAY                           65628 non-null  int8   
 11  CONTINENT                     65628 non-null  object 
 12  max_wind_speed                65628 non-null  float64
 13  av

In [10]:
prep_df.describe().T

,count,mean,std,min,25%,50%,75%,max
reportingYear,65628.0,-35.064957,3.853655,-41.000000,-38.000000,-35.000000,-32.000000,-28.000000
MONTH,65628.0,6.489974,3.450833,1.000000,3.000000,7.000000,9.000000,12.000000
DAY,65628.0,14.517203,8.097332,1.000000,8.000000,14.000000,22.000000,28.000000
max_wind_speed,65628.0,15.515958,3.067272,8.011958,13.324166,15.506820,17.718201,22.991382
avg_wind_speed,65628.0,18.015285,2.310739,14.000100,16.012197,18.020789,20.011702,21.999973
min_wind_speed,65628.0,22.521038,3.059973,15.032589,20.346158,22.540387,24.715251,29.933603
max_temp,65628.0,9.455406,5.216525,-3.141464,5.879821,9.698967,13.282417,20.938266
avg_temp,65628.0,10.448142,5.084529,-0.199176,7.186013,10.701504,14.193578,19.999403
min_temp,65628.0,13.442827,5.216068,0.894827,9.894281,13.692473,17.268000,24.902108
DAY WITH FOGS,65628.0,2.232568,3.778429,0.000000,0.000000,1.000000,2.000000,19.000000


In [11]:
prep_df.describe(exclude=np.number).T

,count,unique,top,freq
countryName,65628,32,United Kingdom,9016
eprtrSectorName,65628,9,Energy sector,24562
EPRTRAnnexIMainActivityLabel,65628,71,Thermal power stations and other combustion in...,21527
FacilityInspireID,65628,7185,https://data.ied_registry.omgeving.vlaanderen....,42
facilityName,65628,7930,Enel Produzione S.p.A.,234
City,65628,5136,--,1975
targetRelease,65628,1,AIR,65628
pollutant,65628,3,Nitrogen oxides (NOX),25982
CONTINENT,65628,1,EUROPE,65628
REPORTER NAME,65628,45016,Michael Brown,25


In [14]:
to_remove_hot_encode = [
    "EPRTRSectorCode",
    "EPRTRAnnexIMainActivityCode",

    "targetRelease",

    "FacilityInspireID",
    "facilityName",
    "City",
]

to_one_hot_encode = [
    "countryName",
    "eprtrSectorName",
    "EPRTRAnnexIMainActivityLabel"
]